In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
from skimage import measure, morphology
import pydicom as dicom
import numpy as np
import cv2
import skimage.transform

#data path 수정
os.chdir("C:\\Users\\User\\Desktop\\G팔로미1\\G-팔로미 data")
os.getcwd()

'C:\\Users\\User\\Desktop\\G팔로미1\\G-팔로미 data'

In [2]:
# Setting base path
#ROOT_PATH = os.path.join(os.getcwd(), 'Dataset_csv')
#CBISDDSM_csvPATH = os.path.join('CBISDDSM')
#MIAS_PATH = os.path.join(ROOT_PATH, 'MIAS')

# Load Dataframe from .csv
calc_train = pd.read_csv(("calc_case_description_train_set.csv"), index_col=0)
calc_test = pd.read_csv(("calc_case_description_test_set.csv"), index_col=0)
mass_train = pd.read_csv(("mass_case_description_train_set.csv"), index_col=0)
mass_test = pd.read_csv(("mass_case_description_test_set.csv"), index_col=0)

In [4]:
#calc_case cropped image path 설정
dcm_image = calc_train['ROI mask file path'].to_list()

dcm_image[:5]

['Calc-Training_P_00005_RIGHT_CC_1/1.3.6.1.4.1.9590.100.1.2.328778919012412769218080124214088709081/1.3.6.1.4.1.9590.100.1.2.393344010211719049419601138200355094682/000000.dcm',
 'Calc-Training_P_00005_RIGHT_MLO_1/1.3.6.1.4.1.9590.100.1.2.67512362210319636108148504382680781938/1.3.6.1.4.1.9590.100.1.2.296281207812130400303493285473798422894/000000.dcm',
 'Calc-Training_P_00007_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.241202057913673145232234613012384759880/1.3.6.1.4.1.9590.100.1.2.314135871111943890422150247820137952041/000000.dcm',
 'Calc-Training_P_00007_LEFT_MLO_1/1.3.6.1.4.1.9590.100.1.2.314250272911170289203882349024229868823/1.3.6.1.4.1.9590.100.1.2.91458279612485515203413781822560852485/000000.dcm',
 'Calc-Training_P_00008_LEFT_CC_1/1.3.6.1.4.1.9590.100.1.2.336811694512764490002272925921108351157/1.3.6.1.4.1.9590.100.1.2.281397494612871934937455783843630775495/000000.dcm',
 'Calc-Training_P_00008_LEFT_CC_2/1.3.6.1.4.1.9590.100.1.2.207558519313356140106496367610041842177/1.3.6.1.4.1.95

In [ ]:
DDSM_dataPATH = './CBIS-DDSM'
images_dir=[]
    
for i in range(len(dcm_image)):
    images_dir.append(os.path.join(DDSM_dataPATH, dcm_image[i]).replace('\n', '').replace('\r', ''))
    
    
images=[]
for i in range(len(images_dir)):
    try:
        d=dicom.read_file(images_dir[i],force=True)
            #d = dicom.dcmread(images_dir[i])
        image=d.pixel_array
        image_small = skimage.transform.resize(image, (224,224), preserve_range=True)
        images.append(image_small)
    except Exception:
        print(i)
        print("error")
            
#np.save('train.npy',images)

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

#images=np.load('train.npy')


X=np.stack(images)[...,None]
X=np.repeat(X,3,axis=-1)
X=(X-np.mean(X))/np.std(X) #standardization
#X=(X-np.min(X))/(np.max(X)-np.min(X)) #normalization

y=calc['pathology']
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
one_hot_y = np_utils.to_categorical(encoded_Y)

y=one_hot_y #BENIGN, BENIGN_WITHOUT_CALLBACK,MALIGNANT순

In [ ]:
np.save('data_X.npy',X)
np.save('data_y.npy',y)